In [1]:
import numpy as np
from enum import IntEnum
from copy import deepcopy
import matplotlib.pyplot as plt
# plt.style.use('seaborn-notebook')
# plt.style.use('seaborn-whitegrid')
import matplotlib.colors as mcolors

In [2]:
class Action(IntEnum):
    up = 0
    right = 1
    down = 2
    left = 3

action_to_str = {
    Action.up : "up",
    Action.right : "right",
    Action.down : "down",
    Action.left : "left",
}

action_to_offset = {
    Action.up : (-1, 0),
    Action.right : (0, 1),
    Action.down : (1, 0),
    Action.left : (0, -1),
}

In [3]:
class GridWorld:

    def __init__(self, height, width, goal, goal_value=5.0, danger=[], danger_value=-5.0, blocked=[], noise=0.0):
        """
        Initialize the GridWorld environment.
        Creates a gridworld like MDP
         - height (int): Number of rows
         - width (int): Number of columns
         - goal (int): Index number of goal cell
         - goal_value (float): Reward given for goal cell
         - danger (list of int): Indices of cells marked as danger
         - danger_value (float): Reward given for danger cell
         - blocked (list of int): Indices of cells marked as blocked (can't enter)
         - noise (float): probability of resulting state not being what was expected
        """

        self._width = width
        self._height = height
        self._grid_values = [0 for _ in range(height * width)] # Initialize state values.
        self._goal_value = goal_value
        self._danger_value = danger_value
        self._goal_cell = goal
        self._danger_cells = danger
        self._blocked_cells = blocked
        self._noise = noise # Noise level in the environment.
        assert noise >= 0 and noise < 1 # Ensure valid noise value.
        self.create_next_values() # Initialize the next state values.


    def reset(self):
        """
        Reset the state values to their initial state.
        """
        self._grid_values = [0 for _ in range(self._height * self._width)]
        self.create_next_values()


    def _inbounds(self, state):
        """
        Check if a state index is within the grid boundaries.
        """
        return state >= 0 and state < self._width * self._height

    def _inbounds_rc(self, state_r, state_c):
        """
        Check if row and column indices are within the grid boundaries.
        """
        return state_r >= 0 and state_r < self._height and state_c >= 0 and state_c < self._width

    def _state_to_rc(self, state):
        """
        Convert a state index to row and column indices.
        """
        return state // self._width, state % self._width

    def _state_from_action(self, state, action):
        """
        Gets the state as a result of applying the given action
        """
        #TO DO:
        #get the row and column from the state
        state_r, state_c = self._state_to_rc(state)

        #get the offset from action to apply to the row and column of the state
        offset_r, offset_c = action_to_offset[action]

        #get the new row and column after applying the offsets 
        new_r, new_c = state_r + offset_r, state_c + offset_c

        # if the new row and column are in bounds
        if self._inbounds_rc(new_r, new_c) and state not in self._blocked_cells:
            #get the new state from the new row and new column
            new_state = new_r * self._width + new_c
            return new_state
        else:
            return state
        

    def is_terminal(self, state):
        """
        Returns true if a state is terminal (goal, or danger)
        """
        #To Do:
        if state == self._goal_cell or state in self._danger_cells:
            return True
        else:
            return False


    def get_states(self):
        """
        Gets all non-terminal states in the environment
        """
        #TO DO:
        #create a an array to store the non-terminal states
        non_terminal_state = []
        #loop through all states in the grid
        for state in range(len(self._grid_values)):
            #if the state is not terminal then add to non_terminal_state array
            if self.is_terminal(state) == False and state not in self._blocked_cells:
                non_terminal_state.append(state)
        #return non_terminal_state array
        return non_terminal_state
    

    def get_actions(self, state):
        """
        Returns a list of valid actions given the current state
        """
        #TO DO:
        #create an array to store valid actions
        if self.is_terminal(state):
            return []
        else:
            return list(Action)
            
    def get_reward(self, state):
        """
        Get the reward for being in the current state
        """
        assert self._inbounds(state)
        # Reward is non-zero for danger or goal
        #TO DO:
        #if the state is in a goal cell (non-zero reward) return the reward value
        if state == self._goal_cell:
            return self._goal_value
        #if the state is in a danger cell (non-zero reward) return the reward value
        elif state in self._danger_cells:
            return self._danger_value
        # else return 0 as there is no reward
        else:
            return 0 
        
    def get_transitions(self, state, action):
        """
        Get a list of transitions as a result of attempting the action in the current state
        Each item in the list is a dictionary, containing the probability of reaching that state and the state itself
        """

        transitions = {}

        new_state = self._state_from_action(state, action)

        if self._noise == 0:
            return [{new_state: 1.0}]  # Deterministic case

        # Compute probability for the intended action
        transitions[new_state] = 1 - self._noise

        # Compute probability for noisy actions
        noise_prob = self._noise / (len(Action) - 1)

        # loop through all actions
        for noise_action in Action:
            # if the noise action is not the same as the action
            if noise_action != action:
                # get the noisy state
                noisy_state = self._state_from_action(state, noise_action)

                # If the noisy state is already in the dictionary, accumulate probability
                if noisy_state in transitions:
                    transitions[noisy_state] += noise_prob
                else:
                    transitions[noisy_state] = noise_prob

        # Return as a list of dictionaries, as expected
        return [{state: prob} for state, prob in transitions.items()]

    def get_value(self, state):
        """
        Get the current value of the state
        """
        assert self._inbounds(state)
        return self._grid_values[state]

    def create_next_values(self):
        """
        Creates a temporary storage for state value updating
        If this is not used, then asynchronous updating may result in unexpected results
        To use properly, run this at the start of each iteration
        """
        #TO DO:
        #create a copy of the grid values
        self._next_values = deepcopy(self._grid_values)



    def set_next_values(self):
        """
        Set the state values from the temporary copied values
        To use properly, run this at the end of each iteration
        """
        # TO DO:
        #set the grid values to the next values
        self._grid_values = self._next_values
        

    def set_value(self, state, value):
        """
        Set the value of the state into the temporary copy
        This value will not update into main storage until self.set_next_values() is called.
        """
        assert self._inbounds(state)
        #TO DO:
        #set the value of the state to the next values
        self._next_values[state] = value


    def solve_linear_system(self, discount_factor=1.0):
        """
        Solve the gridworld using a system of linear equations.
        :param discount_factor: The discount factor for future rewards.
        """
        #To Do:
        #grab every state not just non-teminal states
        # non_terminal_states = self.get_states()
        create_next_values = self.create_next_values()
        states = range(self._height * self._width)
        num_states = len(states)

        # create matrix A representing the coeefficients of the linear system
        A = np.zeros((num_states, num_states))
        # create vector b representing the expected reward
        b = np.zeros(num_states)

        # loop through the states
        for state in states:
            reward = self.get_reward(state)
            # if the state is terminal 
            if self.is_terminal(state):
                # the value is unchanged
                A[state, state] = 1
                # value of the terminal state is the expected reward
                b[state] = self.get_reward(state)
            # if the state is not terminal 
            else:
                # get the actions that the state can take
                valid_actions = self.get_actions(state)

                # loop through the actions that can be taken from state
                for action in valid_actions:
                    # get the transitions that can be made from the state and action
                    transactions = self.get_transitions(state, action)

                    # loop through the transactions
                    for transaction in transactions:
                        # get the new state and the probability of reaching that state
                        new_state, prob = list(transaction.items())[0]
                        total_prob = prob * 1/(len(list(Action)))

                        # update the A matrix and b vector
                        A[state, new_state] -= total_prob * discount_factor
                        # expected reward contribution
                        b[state] += discount_factor * total_prob * self.get_reward(new_state)
            A[state,state] += 1

        # solve the linear equation Ax = b
        values = np.linalg.solve(A, b)

        #update grid world values
        for i, state in enumerate(states):
            self.set_value(state, values[i])
        
        self.set_next_values()

        return self



    def __str__(self):
        """
        Pretty print the state values
        """
        out_str = ""
        for r in range(self._height):
            for c in range(self._width):
                cell = r * self._width + c
                if cell in self._blocked_cells:
                    out_str += "{:>6}".format("----")
                elif cell == self._goal_cell:
                    out_str += "{:>6}".format("GOAL")
                elif cell in self._danger_cells:
                    out_str += "{:>6.2f}".format(self._danger_value)
                else:
                    out_str += "{:>6.2f}".format(self._grid_values[cell])
                out_str += " "
            out_str += "\n"
        return out_str


In [4]:
# Initialize your GridWorld
simple_gw = GridWorld(height=5, width=5, goal=14, danger=[2, 18, 21], blocked=[6, 7, 11, 12], noise=0.0)
# Solve the linear system
values_grid = simple_gw.solve_linear_system(discount_factor=0.95)
print(values_grid)


 -1.70  -2.87  -5.00  -2.34  -0.00 
 -0.90   ----   ----  -0.00   2.34 
 -1.17   ----   ----  -0.00   GOAL 
 -2.87  -3.35  -3.75  -5.00  -0.76 
 -4.69  -5.00  -4.95  -4.64  -2.44 



In [ ]:
def value_iteration(gw, discount, tolerance=0.1):
    #TO DO:

    #initialize the grid values
    gw.create_next_values()
    # keep track of the number of iterations
    iterations = 0

    while True:
        #initialize the delta value
        delta = 0
        iterations += 1
        print(f"iteration: {iterations}")
        # loop through the states of the grid
        for state in gw.get_states():
            # keep track of the value at the state
            old_value = gw.get_value(state)
            # initialize the best value as negative infinity so that we can update it
            best_value = float('-inf')

            # loop through the actions that can be taken from the state
            for action in gw.get_actions(state):
                # initialize the expected value
                expected_value = 0
                # loop through the transactions that can be made from the state 
                for transaction in gw.get_transitions(state, action):
                    # initalize new state and probability of reaching that state
                    new_state, prob = list(transaction.items())[0]
                    # update the expected value using the equation 
                    expected_value += prob * (gw.get_reward(new_state) + discount * gw.get_value(new_state))
                # update the best value with the maximum value between the best value and the expected value
                best_value = max(best_value, expected_value)
                # update the value of the state with the best value
            gw.set_value(state, best_value)
            
            # update the delta value
            delta = max(delta, abs(best_value - old_value))

        gw.set_next_values()
        print(f" Max value Change (delta): {delta}")

        if delta < tolerance:
            print(f"Converged after {iterations} iterations")
            break

In [6]:
# Initialize your GridWorld
simple_gw = GridWorld(height=5, width=5, goal=14, danger=[2, 18, 21], blocked=[6, 7, 11, 12], noise=0.0)
noisy_gw = GridWorld(height=5, width=5, goal=14, danger=[2, 18, 21], blocked=[6, 7, 11, 12], noise=0.2)
discount = 0.95
tolerance = 0.1

In [7]:
value_iteration(simple_gw, discount, 0.1)
print(simple_gw)

iteration: 1
 Max value Change (delta): 5.0
iteration: 2
 Max value Change (delta): 4.75
iteration: 3
 Max value Change (delta): 4.5125
iteration: 4
 Max value Change (delta): 4.286875
iteration: 5
 Max value Change (delta): 4.07253125
iteration: 6
 Max value Change (delta): 3.8689046874999997
iteration: 7
 Max value Change (delta): 3.6754594531249998
iteration: 8
 Max value Change (delta): 3.4916864804687497
iteration: 9
 Max value Change (delta): 3.317102156445312
iteration: 10
 Max value Change (delta): 3.151247048623046
iteration: 11
 Max value Change (delta): 0
Converged after 11 iterations
  3.15   2.99  -5.00   4.51   4.75 
  3.32   ----   ----   4.75   5.00 
  3.49   ----   ----   5.00   GOAL 
  3.68   3.87   4.07  -5.00   5.00 
  3.49  -5.00   4.29   4.51   4.75 



In [8]:
value_iteration(noisy_gw, discount, 0.1)
print(noisy_gw)

iteration: 1
 Max value Change (delta): 4.0
iteration: 2
 Max value Change (delta): 3.249774074074074
iteration: 3
 Max value Change (delta): 2.659602876543209
iteration: 4
 Max value Change (delta): 1.686456533920798
iteration: 5
 Max value Change (delta): 0.7952664756220722
iteration: 6
 Max value Change (delta): 0.5753639243055478
iteration: 7
 Max value Change (delta): 0.438579171940911
iteration: 8
 Max value Change (delta): 0.312768056228189
iteration: 9
 Max value Change (delta): 0.21252354442549043
iteration: 10
 Max value Change (delta): 0.16891374533558698
iteration: 11
 Max value Change (delta): 0.14617127472623723
iteration: 12
 Max value Change (delta): 0.13107707617565817
iteration: 13
 Max value Change (delta): 0.08555986219867542
Converged after 13 iterations
  0.31  -0.11  -5.00   3.56   4.48 
  0.42   ----   ----   4.16   4.86 
  0.51   ----   ----   3.93   GOAL 
  0.60   0.69   1.30  -5.00   4.17 
  0.14  -5.00   2.09   2.90   3.84 



In [9]:
def policy_iteration(gw, discount, tolerance=0.1):
        
    
    

SyntaxError: incomplete input (634266616.py, line 4)